## Observation des Métadonnées

Le catalogue Ecosphères a pour vocation de faciliter l'identification sur chaque territoire des données utiles à l'appui des politiques publiques. Dans le cadre de ces observations, nous nous intéressons à la qualification des données remontées lors d'une recherche à partir des métadonnées disponibles. Nous nous appuierons sur une sélection de métadonnées parmi les champs décrits dans des documents de références tels que la documentation du profile [GeoDCAT-AP](https://semiceu.github.io/GeoDCAT-AP/releases/) et le guide de saisie des éléments de métadonnées [INSPIRE](https://semiceu.github.io/GeoDCAT-AP/releases/).

Les métadonnées recueillies par le POC CKAN Ecosphères à partir des [points de moissonnage](https://github.com/ecolabdata/guichetdonnees-public/blob/main/moissonnages.json) sont à la base de ces observations. Les métadonnées sur les jeux de données sont entreposées dans les tables `package` et `package_extra` de la [base de données CKAN](https://boykoc.github.io/ckan/2019/10/21/ckan-283-database-diagram.html), également disponibles par API suivant le [schéma Ecosphères](https://github.com/ecolabdata/ckanext-ecospheres/blob/main/ckanext/ecospheres/scheming/ecospheres_dataset_schema.yaml) DCAT compatible.

<style type="text/css">
    /* Custom footer, <div> type so positioned as desired. */
    #header_left {
        position: absolute;
        top: 0%;
        left: 2.5%;
        font-size: large;
        font-weight: bold;
    }

    #header_right {
        position: absolute;
        top: 0%;
        right: 2.5%;
        font-size: large;
        font-weight: bold;
    }
</style>

<div id="hidden" style="display:none;">
    <div id="header">
        <p id="header_left">UPDATE</p>
        <p id="header_right">UPDATE</p>
    </div>
</div>

<script src="https://code.jquery.com/jquery-2.2.4.min.js"></script>
<script type="text/javascript">
    var header = $('#header').html();
    if ( window.location.search.match( /print-pdf/gi ) ) {
        Reveal.addEventListener( 'ready', function( event ) {
            $('.slide-background').append(header);
        });
    }
    else {
        $('div.reveal').append(header);
   }
</script>